#### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
    .filter(f"file_date = '{v_file_date}'")

In [0]:
language_df = spark.read.format("delta").load(f"{silver_folder_path}/languages")

In [0]:
movies_languages_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_languages") \
    .filter(f"file_date = '{v_file_date}'")

In [0]:
genres_df = spark.read.format("delta").load(f"{silver_folder_path}/genres")

In [0]:
movies_genres_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_genres") \
    .filter(f"file_date = '{v_file_date}'")

#### Join "languages" y "movies_languages"

In [0]:
languages_mov_lan_df = language_df.join(movies_languages_df, 
                                        language_df.language_id == movies_languages_df.language_id, 
                                        "inner") \
                                  .select(language_df.language_name, movies_languages_df.movie_id, language_df.language_id)

#### Join "genres" y "movies_genres"

In [0]:
genres_mov_gen_df = genres_df.join(
        movies_genres_df, 
        genres_df.genre_id == movies_genres_df.genre_id, 
        "inner") \
    .select(genres_df.genre_name, movies_genres_df.movie_id, genres_df.genre_id) 

#### Join "movies_df", "languages_mov_lan" y "genres_mov_lan_df"

- Filtrar las películas dondes su fecha de lanzamiento sea mayor o igual a 2000

In [0]:
movie_filter_df = movie_df.filter("year_release_date >= 2010")

In [0]:
results_movies_genres_languages_df = movie_filter_df.join(
        languages_mov_lan_df,                                                     movie_filter_df.movie_id == languages_mov_lan_df.movie_id, 
        "inner") \
    .join(
        genres_mov_gen_df, 
        movie_filter_df.movie_id == genres_mov_gen_df.movie_id, 
        "inner")

- Agregar la columna "created_date"

In [0]:
from pyspark.sql.functions import lit

In [0]:
resuls_df = results_movies_genres_languages_df \
    .select(movie_filter_df.movie_id, "title", "duration_time", "release_date", "vote_average", "language_name", "genre_name", "genre_id", "language_id") \
    .withColumn("created_date", lit(v_file_date))

- Ordenar por la columna "release_date" de manera descedente

In [0]:
from pyspark.sql.functions import desc

In [0]:
results_order_by_df = resuls_df.orderBy(resuls_df.release_date.desc())

#### Escribir datos en el DataLake en formato "Delta"

In [0]:
#overwrite_partition_data(results_order_by_df, "movie_gold", "results_movie_genre_language", "created_date")
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.language_id = src.language_id AND tgt.genre_id = src.genre_id AND tgt.created_date = src.created_date'

merge_delta_lake(results_order_by_df, 'movie_gold', 'results_movie_genre_language', gold_folder_path, merge_condition, 'created_date')

In [0]:
%sql
SELECT created_date, COUNT(0) 
FROM movie_gold.results_movie_genre_language
GROUP BY created_date;